<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with a custom metrics provider

This notebook should be run in a Watson Studio project, using **IBM Runtime 24.1 on Python 3.11 XS** runtime environment. **If you are viewing this in Watson Studio and do not see the required runtime env in the upper right corner of your screen, please update the runtime now.**. It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning

## Overview

This sample notebook demonstrates how to configure a custom monitor and compute metrics such as specificity, sensitivity and gender_less40_fav_prediction_ratio for traditional subscriptions. Based on the values specified in the configuration cell, it automatically creates the custom monitor definition, WML batch deployment for the Python function and custom metrics provider with the deployment scoring endpoint. Users must update the appropriate metric computation logic inside the Python function.

During each run, OpenScale invokes the custom metrics provider(python function) and sends inputs like data_mart_id, subscription_id, custom_monitor_id and other parameters. The provider then:

- Reads data from feedback, payload logging, or other datasets.
- Computes and publishes aggregated metrics to the Measurements API.
- Updates the monitor run status to Finished.
  
## Contents

This notebook contains the following parts:

  1. [Set up your environment](#setup)
  1. [Configure values for the custom monitor](#configure_values)
  1. [Create the custom metrics provider - python function](#provider)
  1. [Configure Watson OpenScale](#config)
  1. [Set up the custom monitor](#custom_monitor)
  1. [Get the custom monitor configuration](#get_config)
  1. [Run the custom monitor](#run)
  1. [Risk evaluations for subscription](#evaluate_risk)
     


## 1. Set up your environment <a name="setup"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

### Install the  `ibm_watsonx_ai` and `ibm-watson-openscale` packages.

In [1]:
!pip install --upgrade ibm_watsonx_ai   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

### Action: restart the kernel!

### Credentials for IBM Cloud
To authenticate, in the following code boxes, replace the sample data with your own credentials. Get the information from your system administrator or through the IBM Cloud dashboard.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
############################################################################################
# Paste your credentials into the following section and then run this cell.
############################################################################################
CLOUD_API_KEY = "<Your Cloud IAM API Key>"

In [58]:
#PROJECT_ID = "<Your project id>" #update the project id for pre-production subscription
SPACE_ID = "<Your space id>"
DATAMART_ID =  "<DataMart Id>"
SUBSCRIPTION_ID= "<Subscription Id>"
OPENSCALE_API_URL = "https://api.aiopenscale.cloud.ibm.com"
IAM_URL = "https://iam.cloud.ibm.com/oidc/token"


## 2. Configure values for the custom monitor <a name="configure_values"></a>
Default values for the following custom monitor parameters are set. You can override them by specifying parameter values in the configuration cell.

| Parameter Name                                | Type           | Optional | Description                                                                 | Default Value                              |
|----------------------------------------------|----------------|----------|-----------------------------------------------------------------------------|-------------------------------------------|
| `DEPLOYMENT_NAME`                             | string         | Yes      | Name of the function deployment                                             | `"Custom Metrics Provider Deployment"`   |
| `PYTHON_FUNCTION_NAME`                        | string         | Yes      | Name of the Python function to be deployed                                 | `"Custom Metrics Provider Function"`      |
| `CUSTOM_METRICS_PROVIDER_NAME`                | string         | Yes      | Name for the Custom Metrics Provider                                       | `"Custom Metrics Provider"`               |
| `CUSTOM_MONITOR_NAME`                         | string         | Yes      | Name of the custom monitor                                                 | `"Sample Model Performance"`              |
| `DATAMART_ID`                                 | string         | Yes      | Watson OpenScale DataMart GUID                                             | `"00000000-0000-0000-0000-000000000000"`  |
| `SPACE_ID`                                    | string         | No      | Watson OpenScale Space ID                                                   | `"<Your Space ID>"`  |
| `RUNTIME_ENV`                                 | string         | Yes      | Runtime environment for the Python function                                | `"runtime-24.1-py3.11"`                   |
| `ENABLE_SCHEDULE`                             | boolean        | Yes      | Flag to enable scheduled runs of the monitor                               | `True`                                    |
| `START_TIME`                                  | string         | Yes      | Scheduled run start time (format: `HH:MM:SS`)                              | `"10:00:00"`                              |
| `CUSTOM_METRICS_WAIT_TIME`                    | integer        | Yes      | Time in seconds to check the run status                                    | `60`                                     |
| `DELETE_CUSTOM_MONITOR`                       | boolean        | Yes      | Flag to delete any existing monitor with the same name                     | `True`                                   |
| `DELETE_CUSTOM_MONITOR_INSTANCE`              | boolean        | Yes      | Flag to delete any existing monitor instance                               | `True`                                   |
| `DELETE_INTEGRATED_SYSTEM`                 | boolean        | Yes      | Flag to delete the existing python function and associated custom metric provider                              | `True`                                   |
| `ALGORITHM_TYPES`                              | list[string]   | Yes      | Types of algorithms used (`binary`, `regression`, etc.)                    | `["binary","multiclass","regression","question_answering","summarization","retrieval_augmented_generation","classification","generation","extraction"]`                              |
| `INPUT_DATA_TYPES`                              | list[string]   | Yes      | Type of input data (`structured`, `unstructured`)                          | `["structured","unstructured_text","unstructured_image"]`                          |
| `WOS_URL`                                      | string         | No       | URL of the Watson OpenScale instance                                       | `"https://api.aiopenscale.cloud.ibm.com"` |
| `WML_URL`                                      | string         | No       | URL of Watson Machine Learning instance                                    | `"https://us-south.ml.cloud.ibm.com"`     |
| `CLOUD_API_KEY`                                | string         | No       | IBM Cloud API Key for IAM authentication                                   | `"<Your Cloud IAM API Key>"`             |
| `IAM_URL`                                      | string         | No       | IAM authentication URL                                                     | `"https://iam.ng.bluemix.net/oidc/token"` |
| `SUBSCRIPTION_ID`                              | string         | Yes      | ID of the subscription to be monitored                                     | `"<Subscription Id>"`                    |
| `CUSTOM_MONITOR_METRICS`                       | list[dict]     | No       | List of metric definitions used in the custom monitor                      |                                           |
| └─ `name`                                      | string         | No       | Name of the custom metric (e.g., `sensitivity`)                            |                                           |
| └─ `description`                               | string         | No       | Human-readable description of the metric                                   |                                           |
| └─ `type`                                      | string         | No       | Data type of the metric value (e.g., `number`)                             |                                           |
| `CUSTOM_METRICS_PROVIDER_CREDENTIALS`          | dict           | No       | Dictionary with authentication method for custom metrics provider          |                                           |
| └─ `auth_type`                                 | string         | No       | Authentication method (e.g., `bearer`)                                     |                                           |
| └─ `token_info`                                | dict           | Yes      | Token generation details (used for bearer tokens)                          |                                           |
|     └─ `url`                                   | string         | No       | URL to request IAM token                                                   |                                           |
|     └─ `headers`                               | dict           | No       | HTTP headers for token request                                             |                                           |
| `SCHEDULE              `                       | list[dict]     | No       | List of metric definitions used in the custom monitor                      |                                           |
| └─ `repeat_interval`                           | integer        | No       | Interval between scheduled executions                                      | `1`                                       |
| └─ `repeat_type`                               | string         | No       | Unit of repeat interval (`hour`, `day`, etc.)                              | `"hour"`                                  |
| └─ `delay_unit`                                | string         | No       | Unit of delay duration (`minute`, `second`, etc.)                          |`"minute"`                                 |
| └─ `delay_time`                                | integer        | No       | Delay duration before execution                                            |`5`                                        |
| `CPD_INFO              `                       | list[dict]     | No       | List of metric definitions used in the custom monitor                      |                                           |
| └─ `CPD_URL`                                   | string         | No       | CPD instance URL (if using CPD)                                            |                                           |
| └─ `USERNAME   `                               | string         | No       | CPD Username                                                               |                                           |
| └─ `PASSWORD`                                  | string         | No       | CPD User API Key                                                           |                                           |
| └─ `VERSION`                                   | integer        | No       | Version                                                                    |`5.0`                                      |


### Configuration cell

In [101]:
config = {
  "CLOUD_API_KEY": CLOUD_API_KEY,
  "SPACE_ID": SPACE_ID,
  "DATAMART_ID": DATAMART_ID,
  "SUBSCRIPTION_ID": SUBSCRIPTION_ID,
  "WOS_URL": OPENSCALE_API_URL,
  "CUSTOM_METRICS_WAIT_TIME": 60,
  "CUSTOM_METRICS_PROVIDER_NAME":"Custom Metrics Provider",
  "CUSTOM_MONITOR_NAME":"Sample Model Performance",
  "DEPLOYMENT_NAME": "Custom Metrics Provider Deployment",
  "PYTHON_FUNCTION_NAME": "Custom Metrics Provider Function",
  "MONITOR_METRICS": [
    {
      "name": "specificity",
      "thresholds": {
        "lower_limit": 0.8
      }
    },
    {
      "name": "sensitivity",
      "thresholds": {
        "lower_limit": 0.6,
        "upper_limit": 1.0
      }
    },
    {
      "name": "gender_less40_fav_prediction_ratio",
      "thresholds": {
        "lower_limit": 0.6,
        "upper_limit": 1.0
      }
    }
  ],
  "TAGS": [
      {
          "name": "region",
          "TAG_DESCRIPTION": "Custom metrics tag for monitoring"
      }
  ]
}

## 3. Create the custom metrics provider - Python function <a name="provider"></a>

The Python function receives the required variables, such as the `datamart_id`, `monitor_instance_id`, `monitor_id`, `monitor_instance_parameters` and `subscription_id` from the Watson OpenScale service when it is invoked by the custom monitor. 

In the Python function, add your own logic to compute the custom metrics in the `get_metrics` method, publish the metrics to the Watson Openscale service and update the status of the run to the `finished` state in the custom monitor instance run.

Note: Metric names must exactly match the names defined in the configuration otherwise, an error will occur while publishing the metrics.

In [102]:
#wml_python_function
parms = {
        "url": OPENSCALE_API_URL,
        "iam_url": IAM_URL,
        "apikey": CLOUD_API_KEY
    }
def custom_metrics_provider(parms = parms):
    
    import json
    import requests
    import base64
    from requests.auth import HTTPBasicAuth
    import time
    import uuid
    import datetime
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"

    def get_access_token():
        headers={}
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        headers["Accept"] = "application/json"
        auth = HTTPBasicAuth("bx", "bx")
        data = {
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": parms["apikey"]
        }
        response = requests.post(parms["iam_url"], data=data, headers=headers, auth=auth)
        json_data = response.json()
        access_token = json_data['access_token']
        return access_token    
    
    
    def get_feedback_data(access_token, data_mart_id, feedback_dataset_id):
        json_data = None
        if feedback_dataset_id is not None:
            headers["Authorization"] = "Bearer {}".format(access_token)
            DATASETS_STORE_RECORDS_URL = parms["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&format=list".format(data_mart_id, feedback_dataset_id, 100)
            response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
            json_data = response.json()
            return json_data
    
    #Update the run status to Finished in the Monitor Run
    def update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, run_id, status, error_msg = None):
        monitor_run_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/runs/'+run_id
        completed_timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        patch_payload  = []
        base_path = "/status"
        
        patch_payload.append(get_patch_request_field(base_path, "state", status))
        patch_payload.append(get_patch_request_field(base_path, "completed_at", completed_timestamp))
        if error_msg != None:
            error_json = get_error_json(error_msg)
            patch_payload.append(get_patch_request_field(base_path, "failure", error_json))
        
        headers["Authorization"] = "Bearer {}".format(access_token)
        response = requests.patch(monitor_run_url, headers=headers, json = patch_payload, verify=False)
        monitor_run_response = response.json()
        return response.status_code, monitor_run_response
    
    def get_error_json(error_message):
        trace = str(uuid.uuid4())
        error_json = {
            'trace': trace,
            'errors': [{
                'code': "custom_metrics_error_code",
                'message': str(error_message)
            }]
        }
        return error_json
    
    def get_patch_request_field(base_path, field_name, field_value, op_name="replace"):
        field_json = {
            "op": op_name,
            "path": "{0}/{1}".format(base_path, field_name),
            "value": field_value
        }
        return field_json
    
    #Add your code to compute the custom metrics. 
    def get_metrics(access_token, data_mart_id, subscription_id, feedback_dataset_id):
        #Add the logic here to compute the metrics. Use the below metric names while creating the custom monitor definition
        json_data = get_feedback_data(access_token, data_mart_id, feedback_dataset_id)
        gender_less40_fav_prediction_ratio = 0
        if json_data is not None and len(json_data['records']) > 0:
            fields = json_data['records'][0]['fields']
            values = json_data['records'][0]['values']
            import pandas as pd
            feedback_data = pd.DataFrame(values, columns = fields)
            if 'Sex' in feedback_data.columns:
                female_less40_fav_prediction = len(feedback_data.query('Sex == \'female\' & Age <= 40 & Risk == \'No Risk\''))
                male_less40_fav_prediction = len(feedback_data.query('Sex == \'male\' & Age <= 40 & Risk == \'No Risk\''))
                gender_less40_fav_prediction_ratio = female_less40_fav_prediction / male_less40_fav_prediction
            
        #Remove the tag("region": "us-south") in below metrics while publishing the metric values to Openscale Datamart 
        #if the custom monitor definition is not created with tags
        metrics = {"specificity": 1.2, "sensitivity": 0.85, "gender_less40_fav_prediction_ratio": gender_less40_fav_prediction_ratio, "region": "us-south"}
        #metrics = {"specificity": 1.2, "sensitivity": 0.85, "gender_less40_fav_prediction_ratio": gender_less40_fav_prediction_ratio}
                
        return metrics
        
        
    # Publishes the Custom Metrics to OpenScale
    def publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, feedback_dataset_id, timestamp):
        # Generate an monitoring run id, where the publishing happens against this run id
        custom_metrics = get_metrics(access_token, data_mart_id, subscription_id, feedback_dataset_id)
        measurements_payload = [
                  {
                    "timestamp": timestamp,
                    "run_id": custom_monitoring_run_id,
                    "metrics": [custom_metrics]
                  }
                ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        headers["Content-Type"] = "application/json"
        measurements_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/measurements'
        response = requests.post(measurements_url, headers=headers, json = measurements_payload, verify=False)
        published_measurement = response.json()
     
        return response.status_code, published_measurement
        
    
    def publish( input_data ):

        timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        
        payload = input_data.get("input_data")[0].get("values")
        data_mart_id = payload['data_mart_id']
        subscription_id = payload['subscription_id']
        custom_monitor_id = payload['custom_monitor_id']
        custom_monitor_instance_id = payload['custom_monitor_instance_id']
        custom_monitor_instance_params  = payload['custom_monitor_instance_params']
        custom_monitor_run_id = payload['custom_monitor_run_id']
        payload_dataset_id = payload.get('payload_dataset_id')
        feedback_dataset_id = payload.get('feedback_dataset_id')

        base_url = parms['url'] + '/openscale' + '/' + data_mart_id
        access_token = get_access_token()
        
        published_measurements = []
        error_msgs = []
        run_status = "finished"
        error_msg = None
        
        try:
            status_code, published_measurement = publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitor_run_id, feedback_dataset_id, timestamp)
            if int(status_code) in [200, 201, 202]:
                published_measurements.append(published_measurement)
            else:
                run_status = "error"
                error_msg = published_measurement
                error_msgs.append(error_msg)
                
        except Exception as ex:
            run_status = "error"
            error_msg = str(ex)
            error_msgs.append(error_msg)
            
        finally:
            status_code, response = update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, custom_monitor_run_id, run_status, error_msg)
            if not int(status_code) in [200, 201, 202]:
                error_msgs.append(response)
    
        if len(error_msgs) == 0:
            response_payload = {
                "predictions" : [{ 
                    "values" : published_measurements
                }]

            }
        else:
            response_payload = {
                "predictions":[{
                    "values":[{"errors": error_msgs}]
                }]
            }
        
        return response_payload
        
    return publish
    

## 4. Configure OpenScale. <a name="config"></a>

Import the required libraries and set up the Watson OpenScale Python client.

In [103]:
from ibm_watson_openscale import APIClient
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(
    apikey=config["CLOUD_API_KEY"]
)
wos_client = APIClient(service_url=OPENSCALE_API_URL, authenticator=authenticator, service_instance_id = DATAMART_ID)
wos_client.version

'3.1.2'

## 5. Set up the custom monitor configuration. <a name="custom_monitor"></a>


This setup initializes the WML client, sets the default space, deletes existing resources, and recreates the python function deployment, custom metrics provider, custom monitor definition, and monitor instance.

In [104]:
wos_client.custom_monitor.setup_configuration(config,custom_metrics_provider)

Initialising  Watson Machine Learning (WML) client.
Initilising Cloud WML
Default space set to 192e05be-2fa2-4eda-b377-d32a941fb638
Cleaning up existing deployment Custom Metrics Provider Deployment.
Performing wml_online deployment Cleanup for: Custom Metrics Provider Deployment0199e738-6c4b-7ef4-b115-9c042e463e91
Creating custom function.
Deploy function as ONLINE : Custom Metrics Provider Deployment


######################################################################################

Synchronous deployment creation for id: '4a2f3b7b-132d-4454-a48c-91c66a62357a' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
....
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='257cc752-9f07-4bce-b273-da09f2d6cb

{'function_id': '4a2f3b7b-132d-4454-a48c-91c66a62357a',
 'deployment_id': '257cc752-9f07-4bce-b273-da09f2d6cb11',
 'scoring_url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/257cc752-9f07-4bce-b273-da09f2d6cb11/predictions?version=2025-10-16',
 'integrated_system_id': '0199ec4b-c973-7050-b672-68fdb2b04940',
 'custom_monitor_id': 'sample_model_performance',
 'custom_monitor_instance_id': '0199ec50-9b6a-7b05-a6cf-7a61cd73b51f'}

## 6. Get custom monitor configuration <a name="get_config"></a>

In [105]:
result = wos_client.custom_monitor.get_custom_monitor_configuration(config=config)
result

{'function_id': '4a2f3b7b-132d-4454-a48c-91c66a62357a',
 'deployment_id': '257cc752-9f07-4bce-b273-da09f2d6cb11',
 'scoring_url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/257cc752-9f07-4bce-b273-da09f2d6cb11/predictions?version=2025-10-16',
 'integrated_system_id': '0199ec4b-c973-7050-b672-68fdb2b04940',
 'custom_monitor_id': 'sample_model_performance',
 'custom_monitor_instance_id': '0199ec50-9b6a-7b05-a6cf-7a61cd73b51f'}

In [106]:
custom_monitor_instance_id = result["custom_monitor_instance_id"]

## 7. Run the custom monitor <a name="run"></a>

The cell below runs the custom monitor and publishes the monitor metrics to OpenScale.

In [107]:
#Execute the custom metrics provider deployment
monitor_instance_run_info = wos_client.monitor_instances.run(
        background_mode=False,
        monitor_instance_id=custom_monitor_instance_id
     ).result

monitor_instance_run_info
custom_monitor_run_id = monitor_instance_run_info.metadata.id




 Waiting for end of monitoring run 6a6226f4-ee2c-43ca-900c-1f0934ddb7ff 




finished

---------------------------
 Successfully finished run 
---------------------------




## 8. Risk evaluations for subscription 

The cell below triggers all configured monitors (OOTB and custom) for the selected subscription. It assesses the test data, computes the metrics, and publishes the results to Watson OpenScale and Facts.

For risk assessment of a development-type/pre production subscription, an evaluation dataset must be provided. The risk evaluation function uses the dataset path as an input parameter to evaluate the configured metric dimensions.

Note: Disable Step 7 (Run the custom monitor) and uncomment the code in the following cell to run the custom monitor along with other monitors through MRM to evaluate the risk and publish the results to Facts. 

In [112]:
def get_mrm_monitor_instance():
    monitor_instances = wos_client.monitor_instances.list(data_mart_id = DATAMART_ID, monitor_definition_id = "mrm", target_target_id = SUBSCRIPTION_ID).result.monitor_instances
    if len(monitor_instances) == 1:
        return monitor_instances[0]
    return None


In [ ]:
#mrm_monitor_instance = get_mrm_monitor_instance()
#mrm_monitor_instance_id = mrm_monitor_instance.metadata.id

###################################################################################
#Enable the below code for pre production flow
######################################################################################

#test_data_set_name = "test_data"
#body = {}
#test_data_path= "test_data.csv"

#response = wos_client.monitor_instances.mrm.evaluate_risk(monitor_instance_id=mrm_monitor_instance_id, test_data_set_name=test_data_set_name,
#                                                     test_data_path=test_data_path, body=body, project_id=PROJECT_ID,
#                                                     includes_model_output=True, background_mode=False)
#response.result.to_dict()


#####################################################################################
#Enable the below code for production flow 
######################################################################################
#response  = wos_client.monitor_instances.mrm.evaluate_risk(monitor_instance_id=mrm_monitor_instance_id, 
#                                                      space_id = SPACE_ID, background_mode = False)
#response.result.to_dict()
############################################################################################


## Show custom metrics

In [108]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=custom_monitor_instance_id)

2025-10-16 09:20:19.477150+00:00,sensitivity,0199ec51-fa15-7d8f-8601-2827974b1ca5,0.85,0.6,1.0,['region:us-south'],sample_model_performance,0199ec50-9b6a-7b05-a6cf-7a61cd73b51f,6a6226f4-ee2c-43ca-900c-1f0934ddb7ff,subscription,0199e738-6c4b-7ef4-b115-9c042e463e91
2025-10-16 09:20:19.477150+00:00,gender_less40_fav_prediction_ratio,0199ec51-fa15-7d8f-8601-2827974b1ca5,0.0,0.6,1.0,['region:us-south'],sample_model_performance,0199ec50-9b6a-7b05-a6cf-7a61cd73b51f,6a6226f4-ee2c-43ca-900c-1f0934ddb7ff,subscription,0199e738-6c4b-7ef4-b115-9c042e463e91
2025-10-16 09:20:19.477150+00:00,specificity,0199ec51-fa15-7d8f-8601-2827974b1ca5,1.2,0.8,None,['region:us-south'],sample_model_performance,0199ec50-9b6a-7b05-a6cf-7a61cd73b51f,6a6226f4-ee2c-43ca-900c-1f0934ddb7ff,subscription,0199e738-6c4b-7ef4-b115-9c042e463e91


# [OPTIONAL STEP] Invoke the custom metrics provider python function as part of this notebook.

Run the cell below to validate the custom metrics provider python function by providing the correct parameters to generate the custom metrics.


In [114]:
def get_dataset_id(data_set_type: str):
    data_sets = wos_client.data_sets.list(target_target_id= config["SUBSCRIPTION_ID"], type = data_set_type).result.data_sets
    feedback_data_set_id = None
    if len(data_sets) > 0:
        feedback_data_set_id = data_sets[0].metadata.id
    return feedback_data_set_id

In [115]:
import uuid
parameters = {
    "custom_metrics_provider_id": result["integrated_system_id"],
    "custom_metrics_wait_time":   config["CUSTOM_METRICS_WAIT_TIME"]
}

payload= {
    "data_mart_id" : config["DATAMART_ID"],
    "subscription_id" : config["SUBSCRIPTION_ID"],
    "custom_monitor_id" : custom_monitor_instance_id,
    "custom_monitor_instance_id" : custom_monitor_instance_id,
    "custom_monitor_run_id":custom_monitor_run_id,
    "custom_monitor_instance_params": parameters,
    "feedback_dataset_id": get_dataset_id("feedback")
}

input_data= { "input_data": [ { "values": payload } ]
            }


func_result = custom_metrics_provider()(input_data)
func_result

{'predictions': [{'values': [[{'measurement_id': '0199ec53-dfb8-70db-854c-43e806680dc2',
      'metrics': [{'gender_less40_fav_prediction_ratio': 0,
        'region': 'us-south',
        'sensitivity': 0.85,
        'specificity': 1.2}],
      'run_id': '6a6226f4-ee2c-43ca-900c-1f0934ddb7ff',
      'timestamp': '2025-10-16T09:22:23.800294Z'}]]}]}

## Congratulations

You have finished configuring Custom Monitor Definition and Monitor instance and executing Custom Monitor Run for your Subscription. You can also run the custom monitor from `Watson OpenScale Dashboard`(http://aiopenscale.cloud.ibm.com). Click the tile of your model and select `Evaluate Now` option from `Actions` drop down menu to run the monitor.